In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data= pd.read_csv("/kaggle/input/rossmann-store-sales/train.csv")
store=pd.read_csv("/kaggle/input/rossmann-store-sales/store.csv")
test=pd.read_csv("/kaggle/input/rossmann-store-sales/test.csv")
sample= pd.read_csv("/kaggle/input/rossmann-store-sales/sample_submission.csv")

In [ ]:
data.head()

In [ ]:
store.head()

In [ ]:
test.head()

In [ ]:
sample.head()

In [ ]:
print("store unique value ", data['Store'].nunique())
print("train unique value ", data['Store'].nunique())

In [ ]:
print(data.isnull().sum()/ data.shape[0]*100)
print("-----")
print(round(store.isnull().sum()/ data.shape[0]*100))

In [ ]:
# Duplicates--- No duplicates
data.duplicated().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
# It seems there more outliers in the data with respect to customers

In [ ]:
data.describe() # 5 summary

In [ ]:
data.describe(include="object") # For catergorical varaibles 
# unique- number of distinctive varaible
# freq  - the maximum occurring frequency of the highestvariable in the particular object

In [ ]:
# State Holiday
print(data["StateHoliday"].nunique())
print(data["StateHoliday"].unique())
print(data["StateHoliday"].value_counts())

In [ ]:
##Data_types and checking the null values
data.info()

## Exploratory Data Analysis-- > Data types
1. Categories - StateHoliday, SchoolHoliday, Promo,Open,Date, Store, Dayofweek
2. Numerical  - Sales, Customers
3. Dates    --> Date
    - day,month,year,hour, minute, dayofweek, quater
    - afternoon,morning,evening,night
4. Geographical columns(city) ---> No
    - State, country, province, latitude & longitude
    - avg temperature, population, spending power, gdp
5. Text columns (ex: customer reviews, tweets)---- > No
    - sentiment analysis, Document Term Matrix/ Word embedding(word2vec)
6. Misc -ids, phone numbers, pincode, email-ids  ---> Maybe ID but its required in the submission, Customer is not in the test(We may not consider lets keep it for base model and EDA)[](http://)


In [ ]:
# Understand the data, Process
# Missing values, outliers
# Feature Engineering

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

### Numerical (Univariate)

In [ ]:
data["Customers"].plot(kind="box")
plt.show()
data["Sales"].plot(kind="box")
plt.show()

In [ ]:
sns.distplot(data["Sales"]) # Right Skewed

In [ ]:
sns.distplot(data["Customers"])

### Categorical (Univariate)

In [ ]:
data["DayOfWeek"].value_counts().plot(kind='bar')

In [ ]:
print("total sales value",data.Store.shape)
print("Number of unique values",data.Store.nunique())

In [ ]:
data["Store"].value_counts().value_counts() 
# there are certain missing values(lesser values)

In [ ]:
data["StateHoliday"].value_counts().plot(kind='bar')
# Need to modify the data for better interpretation

In [ ]:
data["Open"].value_counts().plot(kind='bar') #Stores are opened most

In [ ]:
data["SchoolHoliday"].value_counts().plot(kind='bar')

In [ ]:
print("stores are closed sales are  " ,data[data['Open']==0]["Sales"].value_counts())
# When store is closed the sales are zero( Need not worry about in prediction)
print("State Holidays are closed sales are  " ,data[data['StateHoliday']=='a']["Sales"].value_counts()[:10])
print("Public Holidays are closed sales are  " ,data[data['SchoolHoliday']==0]["Sales"].value_counts()[:10])
# But during the state holiday, school holiday there are few sales happening in the stores

In [ ]:
data["Promo"].value_counts().plot(kind='bar')

In [ ]:
print("When the promo is given then the sales are ", data[data['Promo']==1]['Sales'].sum())
print("No promo then the sales are",  data[data['Promo']==0]['Sales'].sum())

### Dates

In [ ]:
data['Date'].min(), data['Date'].max() # We have 7 months data to understand the details

In [ ]:
test['Date'].min(), test['Date'].max() # Test is for the next 2 months

### Bivariate Analysis

In [ ]:
# Intrept the weekly sales of individual store
store_id =data['Store'].unique()[0]
data['Date']= pd.to_datetime(data["Date"],format="%Y-%m-%d")
store_data =data[data["Store"]==store_id]
store_data.resample(on='Date', rule='1w')["Sales"].mean().plot.line(figsize=(20,4))

In [ ]:
## Correlation Analysis
abs(data.corr()).style.background_gradient(cmap="Greens")

In [ ]:
## Box Plot
sns.boxplot(data=data,x="DayOfWeek", y="Sales")

In [ ]:
%timeit pd.to_datetime(data["Date"], format= "%Y-%m-%d")
# time taken to convert date_time 
# full year use "Y" if year presented like (18/19) use small "y"; same for month
# Format(reduces the running time)- small dataset have less influence but for large data set it matters a lot

In [ ]:
def features_dates(data,date_col):
    data[date_col]=pd.to_datetime(data[date_col],format= "%Y-%m-%d")
    data["Day"] =data[date_col].dt.day
    data["month"] =data[date_col].dt.month
    data["year"] =data[date_col].dt.year
    return data
def convert_stateholidays(data):
    data['StateHoliday']=data['StateHoliday'].map({
        "0":0,0:0,'a':1,'b':2,'c':3
        })
    return data


In [ ]:
data_new = features_dates(data,"Date")
data_new =convert_stateholidays(data_new)
data_new

In [ ]:
# Creating the target and the input columns
target_col="Sales"
input_cols = data_new.columns.drop([target_col,'Date'])

In [ ]:
## Heat Map
summary= pd.pivot_table(data=data,index='month',columns='Day',values='Sales',aggfunc='mean')
plt.figure(figsize=(20,6))
sns.heatmap(summary,cmap="Greens")
# In December before the chrismats the sales are more

In [ ]:
sns.scatterplot(data=data,x="Customers",y="Sales")

In [ ]:
store.isna().sum()

In [ ]:
# Preprocessing for getting the datas ready for modelling
def fill_store_data(store):
    store["CompetitionDistance"]= store["CompetitionDistance"].fillna(store['CompetitionDistance'].median())
    store["PromoInterval"] =store['PromoInterval'].fillna(store['PromoInterval'].mode().iloc[0])
    store["CompetitionOpenSinceMonth"]= store['CompetitionOpenSinceMonth'].fillna(0)
    store["CompetitionOpenSinceYear"]= store["CompetitionOpenSinceYear"].fillna(store["CompetitionOpenSinceYear"].mode().iloc[0])
    store["Promo2SinceWeek"]= store['Promo2SinceWeek'].fillna(store["Promo2SinceWeek"].fillna(0))
    store["Promo2SinceYear"]= store['Promo2SinceYear'].fillna(store["Promo2SinceYear"].fillna(store["Promo2SinceYear"].mode().iloc[0])
    return store
                                                             
def feature_engg(df, store):
    df['is_december']= df['month'].apply(lambda v:1 if month==12 else 0)
    return df
    
def preprocess(df,score):
    df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
    df['day']  = df.Date.day
    df['month']=df.Date.month
    df['year']=df.Date.year
    df= feature_engg(df,store)
    return df
    

In [ ]:
# Base Model 
# Decision Tree Regressor
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
X_train, X_test, y_train,y_test= train_test_split(
                         data_new[input_cols],data_new[target_col],
                         test_size=0.2,random_state=1)

In [ ]:
model = DecisionTreeRegressor(max_depth=5,random_state=1)
model.fit(X_train,y_train)
y_test_pred= model.predict(X_test)
mse = mean_squared_error(y_test,y_test_pred)
rmse = np.sqrt(mse)
r2_score= r2_score(y_test,y_test_pred)
print ("MSE Score",mse)
print ("RMSE Score", np.sqrt(mse))
print( "R2 score", r2_score)

In [ ]:
from sklearn.ensemble import RandomForestRegressor 
model = RandomForestRegressor(max_depth=5,random_state=1)
model.fit(X_train,y_train)

In [ ]:
# Random Forest Regressor
y_test_pred= model.predict(X_test)
mse = mean_squared_error(y_test,y_test_pred)
rmse = np.sqrt(mse)
#r2_score= r2_score(y_test,y_test_pred)
print ("MSE Score",mse)
print ("RMSE Score", np.sqrt(mse))
#print( "R2 score", r2_score)

In [ ]:
# Ofcourse the Random Forest Regressor has lesser MSE compared to Decision Tree

In [ ]:
# Creating a base decision tree classfier to understand the gini classification and the information  gain from the cuts
X= data[input_cols]
y= data[target_col]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
giniDecisionTree = DecisionTreeRegressor(criterion='mse',random_state = 100,max_depth=3, min_samples_leaf=5)
giniDecisionTree.fit(X, y)

In [ ]:
!pip install pydotplus

In [ ]:
from IPython.display import Image  
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
import pydotplus

features = X.columns
# Create DOT data
dot_data = export_graphviz(giniDecisionTree, out_file=None, feature_names=features)

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)  

# Show graph
Image(graph.create_png())

In [ ]:
# To identify every cuts in the data set
col =' Customers'
cuts1 = X_train['Customers'].sort_values().unique().tolist()
cuts2 = cuts1[1:]
cuts =[]
for i in range (len(cuts2)):
    cuts.append(np.mean([cuts1[i],cuts2[i]]))

In [ ]:
samples_left = X_train[X_train['Customers']< 424.5]
samples_left

In [ ]:
# More Feature Engineering
# Hyper parameter tuning with cross validation result